In [1]:
import pandas as pd
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
import time
import os
import pickle
from torch.utils.data import TensorDataset, DataLoader  
from modules.utils import load_cv, norm_data, get_class_weights
from modules.eval_funcs import cv_fold_eval
from modules.cleaning import rr_prefix
from modules.models import FeedFwdNN, MlpModel
from modules.lr_scheduler import build_scheduler
from modules.tab_train import train_IBP_model, EarlyStopper, test_eval, pycyto_norm
from torch.optim.lr_scheduler import CosineAnnealingLR
from timm.scheduler.cosine_lr import CosineLRScheduler

rand_seed = 42
moa_dict = {'PI3K' : 0, 'p38 MAPK': 1, 'RAF': 2, 'AURK': 3, 'CDK': 4, 'EGFR': 5, 'ROCK': 6,
             'MEK': 7, 'GSK': 8, 'mTOR': 9}

# Load Data:

In [2]:
ki_ibp = pd.read_csv('data/ibp/ki_ibp.csv')
print(ki_ibp.shape)
ki_ibp.head(2)

(635, 4778)


,Metadata_Source,Metadata_Plate,Metadata_Well,Cells_AreaShape_Area,Cells_AreaShape_BoundingBoxArea,Cells_AreaShape_BoundingBoxMaximum_X,Cells_AreaShape_BoundingBoxMaximum_Y,Cells_AreaShape_BoundingBoxMinimum_X,Cells_AreaShape_BoundingBoxMinimum_Y,Cells_AreaShape_Center_X,...,smiles,clinical_phase,moa_src,Metadata_JCP2022,Metadata_InChIKey,Metadata_PlateType,blur_score,sat_score,focus_score,comp_score
0,source_3,JCPQC023,G14,3227.817708,5310.328125,589.21875,541.552083,519.484375,471.942708,553.446757,...,Nc1cc(c(cn1)-c1cc(nc(n1)N1CCOCC1)N1CCOCC1)C(F)...,Phase 3,dr_hub,JCP2022_013856,CWHUFRVAEUJCEF-UHFFFAOYSA-N,TARGET2,0.430742,0.453621,0.517562,1.401925
1,source_4,BR00121424,G14,4255.300000,7338.300000,572.70000,554.500000,488.420000,470.490000,530.260000,...,Nc1cc(c(cn1)-c1cc(nc(n1)N1CCOCC1)N1CCOCC1)C(F)...,Phase 3,dr_hub,JCP2022_013856,CWHUFRVAEUJCEF-UHFFFAOYSA-N,TARGET2,0.436727,0.144924,0.386009,0.967661


In [5]:
chem_keys = ki_ibp.Metadata_InChIKey.unique()

In [6]:
# Specify the file path
file_path = "chem_keys.txt"

# Open the file in write mode and write each string on a new line
with open(file_path, 'w') as file:
    for string in chem_keys:
        file.write(string + '\n')

# Load CV Fold Data:

In [3]:
# Load CV split data:
cv_splits = 5
cv_path = 'data/cv_splits/'
cv_data = load_cv(cv_path, cv_splits, ki_ibp, moa_dict, norm=None)

In [7]:
# Load Spherized and Harmonized Data:
pycy_spher = 'output/MAD_Sphere/MADS_PyCyFS.pkl'
shap_spher = 'output/MAD_Sphere/MADS_ShapFS.pkl'
pycy_harm = 'output/MAD_Harmony/MAD_Harmony_PyCyFS.pkl'
shap_harm = 'output/MAD_Harmony/MAD_Harmony_ShapFS.pkl'

with open(shap_spher, 'rb') as file:
    loaded_data = pickle.load(file)
    
# Specify the prefix to remove
prefix_to_remove = 'sph_'
loaded_data = rr_prefix(loaded_data, prefix_to_remove)

In [5]:
# Load Feature Selection Data:
cv_results = pd.read_csv('output/feat_sel/shap_fs_res.csv')

# Kaggle MLP Model:
Implementing Pytorch, multiclass, single-input version of the Kaggle-winning MOA prediction model, code available at:

https://github.com/guitarmind/kaggle_moa_winner_hungry_for_gold/blob/main/final//Best%20LB/Training/2heads-ResNest-train.ipynb

**Essential Model Characteristics:**
- 6 fully connected layers with batch-norm.
- Drop out in first two hidden layers.
- Combinations of 'elu' (first, third), 'relu' (second, third), and 'selu' activation functions (fifth layer).
- Adam optimizer.
- Reduce lr on plateau by a factor of 0.5 with patience of 4.

**Model imported from modules.models**

## Static Variables:

In [43]:
# Model parameters:
num_classes = 10
n_epochs = 95
batch_size = 16
n_neurons = 2048
lr = 5e-6
device = 'cpu'
sched_type = 'linear'
run_dir = 'output/IBP_MLP/MADS_SHAP/'
if not os.path.exists(run_dir):
        os.makedirs(run_dir)
early_stop = 95
opt_wd = 1e-8
norm = True
dropout = 0.2
eq_weights = False
full_train = False
tab_norm = 'mads'
val_criteria = 'loss'

# Shapely Features:
n_features = loaded_data[0]['X_train'].shape[1] # 150 #len(cv_data[0]['X_train'].columns)
top_feats = cv_results['features'][0:n_features].tolist()

In [44]:
class Args:
    def __init__(self, num_classes, n_epochs, batch_size, n_neurons, lr, sched_type, 
                 n_features, opt_wd, norm, dropout, eq_weights, full_train, tab_norm):
        self.num_classes = num_classes
        self.n_epochs = n_epochs
        self.batch_size = batch_size
        self.n_neurons = n_neurons
        self.lr = lr
        self.sched_type = sched_type
        self.t_initial = 8
        self.cycle_mul = 2
        self.min_lr = 5e-6
        self.lrd_fac = 0.5
        self.warmup_epochs = 10
        self.warmup_lr = 5e-6
        self.lr_restarts = 5
        self.lr_pat = 6
        self.workers = 1
        self.model_type = 'MLP'
        self.n_features = n_features
        self.opt_wd = opt_wd
        self.norm = norm
        self.dropout = dropout
        self.eq_weights = eq_weights
        self.full_train = full_train
        self.tab_norm = tab_norm
        self.val_criteria = val_criteria

args = Args(num_classes, n_epochs, batch_size, n_neurons, lr, sched_type, 
            n_features, opt_wd, norm, dropout, eq_weights, full_train, tab_norm, val_criteria)

with open('{}/args_vars.txt'.format(run_dir), 'w') as file:
    for key, value in vars(args).items():
        file.write(f"{key}: {value}\n")

# Model Training Loop:

In [ ]:
cv_best_mods = []
acc_scores = []
f1_scores = []
res_dict = {}

# Lists to hold best results from CV folds:
cv_train_loss = []
cv_train_acc = []
cv_val_loss = []
cv_val_acc = []
cv_cpnd_acc = []

for i, fold in enumerate(loaded_data):
    best_cv_mod = 0
    
    # Create folder for training output:
    train_dir = os.path.join(run_dir, 'CV%s_Output/' % i)
    if not os.path.exists(train_dir):
        os.makedirs(train_dir)

    
    # DEFINE MODEL, OPTIMIZER, CRITERION AND SCHEDULER:
    args.n_features = loaded_data[i]['X_train'].shape[1] # variable number of features selected by pycy per fold
    model = FeedFwdNN(args.n_features, args.num_classes, args.n_neurons, args.dropout)
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=args.opt_wd)
    
    if args.eq_weights:   # Equal Weight Loss Function
        class_weights = get_class_weights(fold['y_train'])
        criterion = nn.CrossEntropyLoss(weight=class_weights)
    else:
        criterion = nn.CrossEntropyLoss()
        
    lr_scheduler = build_scheduler(args, optimizer)
    early_stopper = EarlyStopper(patience=early_stop, min_delta=1e-5)
    
    # Extract cv-fold training and validation sets:
    if tab_norm == 'minmax':
        cv_X_train = fold['X_train'].astype(np.float32)[top_feats].reset_index(drop=True)
        cv_X_val = fold['X_val'].astype(np.float32)[top_feats].reset_index(drop=True)
        cv_X_test = fold['X_test'].astype(np.float32)[top_feats].reset_index(drop=True)
        cv_y_train = fold['y_train']
        cv_y_val = fold['y_val']
        cv_y_test = fold['y_test']
        val_meta = fold['val_meta'].Metadata_JCP2022.to_list()
        test_meta = fold['test_meta'].Metadata_JCP2022.to_list()

        # Combine training and validation sets if using full training set:
        if args.full_train:
            cv_X_train = pd.concat([cv_X_train, cv_X_val]).reset_index(drop=True)
            cv_y_train = np.concatenate((cv_y_train, cv_y_val))       
    
        # Normalize Feature Data:
        if args.norm:
            cv_X_train, cv_X_val, cv_X_test = norm_data('minmax', cv_X_train, cv_X_val, cv_X_test)
        
        # Train Model:
        vl_mod, va_mod, mdl_list = train_IBP_model(i, cv_X_train, cv_X_val, cv_y_train, cv_y_val, val_meta,
                                     model, optimizer, criterion, device, train_dir, args,
                                     lr_scheduler, early_stopper)
    
    elif tab_norm == 'mads':
        cv_X_train = loaded_data[i]['X_train'].astype(np.float32)
        cv_X_test = loaded_data[i]['X_test'].astype(np.float32)
        cv_y_train = loaded_data[i]['y_train'].values
        cv_y_test = loaded_data[i]['y_test'].values
        test_meta = loaded_data[i]['test_meta'].Metadata_JCP2022.to_list()
       
        # Train Model (Note no separate val. set, combined):
        vl_mod, va_mod, mdl_list = train_IBP_model(i, cv_X_train, cv_X_train, cv_y_train, cv_y_train, val_meta,
                                     model, optimizer, criterion, device, train_dir, args,
                                     lr_scheduler, early_stopper)
    
    
    # TEST SET EVALUATION:
    print("___________TEST EVALUATION - CV %s____________" % i)
    if val_criteria == 'loss':
        print("--------- VLoss Model Metrics ---------")
        acc, f1, _ = test_eval(i, vl_mod, cv_X_test, cv_y_test, test_meta, device, 
                                     train_dir, 'MLP')
    elif val_criteria == 'acc':
        print("--------- VAcc Model Metrics ---------")
        acc, f1, _ = test_eval(i, va_mod, cv_X_test, cv_y_test, test_meta, device, 
                                     train_dir, 'MLP')
    else:
        print("--------- Final Model Metrics ---------")
        acc, f1, _ = test_eval(i, mdl_list[-1], cv_X_test, cv_y_test, test_meta, device, 
                                       train_dir, 'MLP')
    print("---------------------------------------\n")
    
    acc_scores.append(acc)
    f1_scores.append(f1)
    
    
# Updating Results Dictionary:
res_dict['CV Accuracy'] = acc_scores
res_dict['CV F1 Score'] = f1_scores
res_dict['Average Accuracy'] = np.mean(acc_scores)
res_dict['Average F1 Score'] = np.mean(f1_scores)
    
# Save Results:    
with open('{}/results_{:.2f}_Acc.txt'.format(run_dir, res_dict['Average Accuracy']), 'w') as file:
    for key, value in res_dict.items():
        file.write(f"{key}: {value}\n")